In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local", appName="transformacionesYAcciones")

In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=transformacionesYAcciones>

In [6]:
!ls /home/sparky/Documentos/curso-apache-spark-platzi/files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [7]:
path = "/home/sparky/Documentos/curso-apache-spark-platzi/files/"

equiposOlimpicosRDD = sc.textFile(path+"paises.csv") \
    .map(lambda line : line.split(","))


In [8]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [9]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [10]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey() \
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [12]:
equiposArgentinosRDD = equiposOlimpicosRDD \
    .filter(lambda l : "ARG" in l)
equiposArgentinosRDD.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [13]:
equiposOlimpicosRDD.count()

1185

In [14]:
equiposOlimpicosRDD.countApprox(20)

1185

In [15]:
!ls /home/sparky/Documentos/curso-apache-spark-platzi/files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [16]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv") \
    .map(lambda l : l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path+"deportista2.csv") \
    .map(lambda l : l.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

In [17]:
deportistaOlimpicoRDD.count()

135572

In [18]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [19]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [20]:
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l : (l[-1],l[:-1])) \
    .join(equiposOlimpicosRDD.map(lambda x : (x[0],x[2])))

In [21]:
resultado = sc.textFile(path+"resultados.csv") \
    .map(lambda l:l.split(","))

In [22]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])

In [23]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [24]:
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [26]:
deportistaPaises.join(resultadoGanador).take(5)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold'))]

In [27]:
valoresMedallas = {
    'Gold' : 7,
    'Silver' : 5,
    'Bronze' : 4
}

In [29]:
paisesMedallas = deportistaPaises.join(resultadoGanador)

In [33]:
paisesMedallas = paisesMedallas \
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1]]))

In [34]:
from operator import add
conclusionRDD = paisesMedallas.reduceByKey((add)) \
    .sortBy(lambda x : x[1],ascending = False)

In [35]:
conclusionRDD.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [36]:
sc.stop()